In [0]:

from pyspark.sql.functions import *
from pyspark.sql import functions as f

from pyspark.sql.types import *
import requests,json,time

In [0]:
#define a function for reading API data
def read_api(url : str):
    normalized_data = dict()
    data = requests.get(url).json() 
    normalized_data = data 
    return json.dumps(normalized_data)

In [0]:
#defining schema 
df_schema=StructType([
  StructField('results',ArrayType(
       StructType([
         
          StructField('user',StructType([
                StructField('gender',StringType(),True),
                StructField('name',StructType([
                      StructField('title',StringType(),True),
                      StructField('first',StringType(),True),
                      StructField('last',StringType(),True)
                    ])),
                StructField('location',StructType([
                    StructField('street',StringType(),True),
                    StructField('city',StringType(),True),
                    StructField('state',StringType(),True),
                    StructField('zip',LongType(),True)
                    ])) ,
               StructField('picture',StructType([
                      
                    StructField('large',StringType(),True),
                    StructField('medium',StringType(),True),
                    StructField('thumbnail',StringType(),True)
               ])),   
                    
                StructField('email',StringType(),True),
                StructField('username',StringType(),True),
                StructField('password',StringType(),True),
                StructField('salt',StringType(),True),
                StructField('sha1',StringType(),True),
                StructField('md5',StringType(),True),
                StructField('sha256',StringType(),True),
                StructField('registered',LongType(),True),
                StructField('dob',LongType(),True),
                StructField('phone',StringType(),True),
                StructField('cell',StringType(),True)
                ]))
         ])
  ) ) ,
  StructField('nationality',StringType(),True),
  StructField('seed',StringType(),True),
  StructField('version',StringType(),True)
])

In [0]:
url = "https://randomuser.me/api/0.8/?results=1"
while True:
      payload = json.loads(read_api(url))
      #create a rdd from json    
      payload_rdd = spark.sparkContext.parallelize([payload])
        
      #create a dataframe from rdd with schema 
      df = spark.read.schema(df_schema).json(payload_rdd)
      # Add timestamp and write the dataframe in external path.
      df1=df.withColumn("curr_timestamp", current_timestamp())\
          .write.format("delta").mode("append").save("dbfs:/api_jan_bronze")
